In [1]:
import numpy as np
import sys 


from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy_sp import *
from sympy.physics.mechanics import *
from sympy import *
from Data_generator_py_active import image_process
import sympy
import torch
import sys
import HLsearch as HL
import example_pendulum
import time

(1, 500, 51, 51)


In [2]:
save = False
environment = "server"
sample_size = 10
device = 'cuda:5'

In [3]:
sys.path.append(r'../../../HLsearch/')
#Saving Directory
params = {}
params['adding_noise'] = False
params['noise_type'] = 'image_noise'
params['noiselevel'] = 1e-1
params['changing_length'] = False
params['c'] = 0
params['g'] = 9.81
params['l'] = 1
if environment == 'laptop':
    root_dir =R'C:\Users\87106\OneDrive\sindy\progress'
elif environment == 'desktop':
    root_dir = R'E:\OneDrive\sindy\progress'
elif environment == 'server':
    root_dir = R'/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression'
x,dx,ddx,tau = image_process(sample_size,params)
X = []
Xdot = []
for i in range(len(x)):
    temp_list = [float(x[i]),float(dx[i])]
    X.append(temp_list)
    temp_list = [float(dx[i]),float(ddx[i])]
    Xdot.append(temp_list)
X = np.vstack(X)
print(X.shape)
Xdot = np.vstack(Xdot)

(4998, 2)


In [4]:
states_dim = 2
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

states are: (x0, x0_t)
states derivatives are:  (x0_t, x0_tt)


In [5]:
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

In [6]:
#build function expression for the library in str
expr= HL.buildFunctionExpressions(2,states_dim,states,use_sine=True)
#expr=['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
"a list of candidate function"
print(expr)
expr.pop(5)

#library function expression for the dissipation function in str
d_expr = [ 'x0_t','x0_t**2','x0_t**3']
#d_expr = ['x0_t**2']

['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


In [7]:
Zeta, Eta, Delta, Dissip = LagrangianLibraryTensor(X,Xdot,expr,d_expr,states,states_dot,device,scaling=True)
Eta = Eta.to(device)
Zeta = Zeta.to(device)
Delta = Delta.to(device)
Dissip = Dissip.to(device)

d_qdot [1, 2*x0_t, 3*x0_t**2]
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:5') cuda:5
tensor([-0.0787,  0.3140,  0.7092,  ...,  3.5551,  3.8939,  4.4527],
       device='cuda:5', dtype=torch.float64) cuda:5
tensor([4.6504e-03, 7.3963e-02, 3.7725e-01,  ..., 9.4790e+00, 1.1372e+01,
        1.4870e+01], device='cuda:5', dtype=torch.float64) cuda:5


In [8]:
#coefficients for the Lagrangian
mask = torch.ones(len(expr),device=device)
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()
# coefficients for the dissipation function
d_mask = torch.ones(len(d_expr),device=device)
xi_d = torch.zeros(len(d_expr),device=device)
prevxi_d = xi_d.clone().detach()

In [9]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [10]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

In [11]:
def proxL1norm(w_hat, alpha):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    return w

In [12]:
def training_loop(coef,d_coef, prevcoef, predcoef, Zeta, Eta, Delta,Dissip, xdot,tau, bs, lr, lam,d_training):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]
    momentum = True
    #if(torch.is_tensor(xdot)==False):
        #xdot = torch.from_numpy(xdot).to(device).float()
    v = coef.clone().detach().to(device).requires_grad_(True)
    v_d = d_coef.clone().detach().to(device).requires_grad_(True)
    prev = prevcoef.clone().detach().to(device).requires_grad_(True)
    prev_d = predcoef.clone().detach().to(device).requires_grad_(True)
    q_tt_pred = 2
    for i in range(tl//bs):
        #computing acceleration with momentum
        #if (momentum == True):
        #vhat = (v + ((i - 1) / (i + 2)) * (v - prev)).clone().detach().requires_grad_(True)
        #vdhat = (v_d + ((i - 1) / (i + 2)) * (v_d - prev_d)).clone().detach().requires_grad_(True)
        vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
        vdhat = v_d.requires_grad_(True).clone().detach().requires_grad_(True)
        prev = v
        prev_d = v_d
        #Computing loss
        zeta = Zeta[:,i*bs:(i+1)*bs]
        eta = Eta[:,i*bs:(i+1)*bs]
        delta = Delta[:,i*bs:(i+1)*bs]
        dissip = Dissip[:,i*bs:(i+1)*bs]
        x_t = xdot[i*bs:(i+1)*bs,:]
        #forward
        tau_pre = tauforward(vhat,vdhat,zeta,eta,delta,dissip,x_t,device)
        tau_true = tau[i*bs:(i+1)*bs]
        tau_true = torch.from_numpy(tau_true).to(device).float()
        lossval = loss(tau_pre, tau_true)
        lossval.requires_grad_(True)
        #Backpropagation
        lossval.backward()
        with torch.no_grad():
            v = vhat - lr * vhat.grad
            #v = proxL1norm(v, lr * lam)
            if d_training == True:
                v_d = vdhat - lr * vdhat.grad
                #v_d = proxL1norm(v_d, lr * lam)
            # Manually zero the gradients after updating weights
            vhat.grad = None
            vdhat.grad = None
        loss_list.append(lossval.item())
    return v, v_d, prev, prev_d, torch.tensor(loss_list).mean().item(),tl

In [13]:
Epoch = 100
i = 1
lr = 1e-3
lam = 0.1
temp = 1000
d_training = False
while(i<=Epoch):
    xi_L , xi_d, prevxi_L, prevxi_d, lossitem, q= training_loop(xi_L,xi_d,prevxi_L,prevxi_d,Zeta,Eta,Delta,Dissip,Xdot,tau,128,lr,lam,d_training)
    if i %20 == 0:
        print("\n")
        print("Stage 1")
        print("Epoch "+str(i) + "/" + str(Epoch))
        print("Learning rate : ", lr)
        print("Average loss : " , lossitem)
    if(temp <=5e-3):
        break
    if(temp <=1e-1):
        lr = 1e-5
    temp = lossitem
    i+=1



Stage 1
Epoch 20/100
Learning rate :  0.001
Average loss :  13.053908348083496


Stage 1
Epoch 40/100
Learning rate :  0.001
Average loss :  4.3146891593933105


Stage 1
Epoch 60/100
Learning rate :  0.001
Average loss :  2.303388833999634


Stage 1
Epoch 80/100
Learning rate :  0.001
Average loss :  1.5803558826446533


Stage 1
Epoch 100/100
Learning rate :  0.001
Average loss :  1.248397707939148


In [14]:
threshold = 1e-2
threshold_d = 1e-10
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()
#surv_index_d = ((torch.abs(xi_d) >= threshold_d)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
#d_expr = np.array(d_expr)[surv_index_d].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
#xi_d =xi_d[surv_index_d].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()
prevxi_d = xi_d.clone().detach()
mask = torch.ones(len(expr),device=device)

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
xi_dcpu = np.around(xi_d.detach().cpu().numpy(),decimals=2)
L = HL.generateExpression(xi_Lcpu,expr)
D = HL.generateExpression(xi_dcpu,d_expr)
print(len(xi_L))

print(D)

13
9.72*x0_t + 7.91*x0_t**2 + 4.69*x0_t**3


In [16]:
for stage in range(1000):

    #Redefine computation after thresholding
    Zeta, Eta, Delta, Dissip = LagrangianLibraryTensor(X,Xdot,expr,d_expr,states,states_dot,device,scaling=True)
    Eta = Eta.to(device)
    Zeta = Zeta.to(device)
    Delta = Delta.to(device)
    Dissip = Dissip.to(device)

    #Training
    Epoch = 200
    i = 1
    lr = 1e-4
    if(stage==1):
        lam = 0
    else:
        lam = 0.1
    temp = 1000
    while(i<=Epoch):   
        xi_L , xi_d, prevxi_L, prevxi_d, lossitem, q= training_loop(xi_L,xi_d,prevxi_L,prevxi_d,Zeta,Eta,Delta,Dissip,Xdot,tau,128,lr,lam,d_training)
        if i %50 == 0:
            print("\n")
            print("Stage ",stage)
            print("Epoch "+str(i) + "/" + str(Epoch))
            print("Learning rate : ", lr)
            print("Average loss : " , lossitem)
        temp = lossitem
#       if temp <= 0.7:
#            lr = 5e-7
        if(temp <=1e-6):
            break
        i+=1
    
    ## Thresholding small indices ##
    threshold = 2e-1
    threshold_d = 1e-3
    surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    expr = np.array(expr)[surv_index].tolist()
    #surv_index_d = ((torch.abs(xi_d) >= threshold_d)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    #d_expr = np.array(d_expr)[surv_index_d].tolist()
    if stage > 60:
        d_training = True
    xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
    #xi_d =xi_d[surv_index_d].clone().detach().requires_grad_(True)
    prevxi_L = xi_L.clone().detach()
    prevxi_d = xi_d.clone().detach()
    mask = torch.ones(len(expr),device=device)

    ## obtaining analytical model
    xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
    xi_dcpu = np.around(xi_d.detach().cpu().numpy(),decimals=2)
    L = HL.generateExpression(xi_Lcpu,expr)
    D = HL.generateExpression(xi_dcpu,d_expr)
    print("expression length:\t",len(xi_L))
    print("Result stage " + str(stage+2) + ":" , L)
    print("Dissipation function:", D)
    equation_motion(xi_Lcpu,xi_dcpu,expr,d_expr,states,states_dot)

d_qdot [1, 2*x0_t, 3*x0_t**2]
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:5') cuda:5
tensor([-0.0787,  0.3140,  0.7092,  ...,  3.5551,  3.8939,  4.4527],
       device='cuda:5', dtype=torch.float64) cuda:5
tensor([4.6504e-03, 7.3963e-02, 3.7725e-01,  ..., 9.4790e+00, 1.1372e+01,
        1.4870e+01], device='cuda:5', dtype=torch.float64) cuda:5


Stage  0
Epoch 50/200
Learning rate :  0.0001
Average loss :  0.5934061408042908


Stage  0
Epoch 100/200
Learning rate :  0.0001
Average loss :  0.5929437875747681


Stage  0
Epoch 150/200
Learning rate :  0.0001
Average loss :  0.5924872159957886


Stage  0
Epoch 200/200
Learning rate :  0.0001
Average loss :  0.5920388102531433
expression length:	 9
Result stage 2: 2.82*x0 + 0.33*x0_t + -7.62*sin(x0) + 7.3*cos(x0) + 9.24*x0**2 + -2.63*sin(x0)**2 + 9.21*x0*cos(x0) + -2.66*sin(x0)*cos(x0) + -4.61*cos(x0)**2
Dissipation function: 9.72*x0_t + 7.91*x0_t**2 + 4.37*x0_t**3
Delta 2.82*1 + -7.62*cos(x0) + 7.3*-sin(x0) + 9.24*2*x0 + -2.63*2*si